In [1]:
!pip install feature_engine==1.6.0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sb
# Import essential libraries
import os
import joblib
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
from numpy.random import rand
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from random import randrange
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
import xgboost as xg
from helper import *
from geneticHelper import *
from statHelper import *
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

In [3]:
# mainRes=GeneticAlgo(Data.copy(),class_name='class')
# mainRes.result

## Genetic algorithm Version 2 <br>
__initilization of population:__ <br>
population contains all the cromosome. On the initilization of population, the size of population and the number of feature is given. Here total "size" cromosome is generated where each cromosome has n_feature. there, 30% of the cromosome set to false initially. Then a random shuffle happends to mix them. <br>
The initial population is for the first generation. <br><br>
__Fitness Score:__ <br>
On the Fitness score, the population is passed. A random Forest Classifier model with n_estimators=200 and random_state=0 is used. The score is calculated for each chromosome in the population. Finally the score and population is sorted based on accuracy score and returns from the fitnes Score function. <br>
<br>
__Selection:__ <br>
Population sorted based on accuracy and the number of parents n_parents is passed here. It takes the top n_parents cromosome and based on accuracy score and return it. 
<br><br>
__CrosOver:__  <br>
In crossover, the selected n_parents cromosome is passed to crossOver function. It take every two cromosome, merge first half of the first cromosome, and second half from the second cromosome. then add the new cromosome to the population.
<br> <br>
__Mutation:__ <br>
It calculate the number of feature to be muted based on the mutation rate. Then it take every cromosome from the population and randomly choose the indexes to me muted. Then it change the feature, if true then false, if false then true. <br>
<br>
The final population will passed to the next generation.

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from random import randint
class storeData:
    def __init__(self, result, features, cromosome,data):
        self.result = result
        self.features = features
        self.cromosome = cromosome
        self.data=data

    def __repr__(self):
        return f"MyDataContainer(result={self.result}, features={self.features}, cromosome={self.cromosome},data={self.data})"

def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=bool)  # Initialize n_feat sized array with True/One     
        chromosome[:int(0.3*n_feat)]=False       # Set first 30% of the elements False/Zero
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population,X_train,Y_train,X_test,Y_test):
    scores = []
    logmodel = RandomForestClassifier(n_estimators=200, random_state=0)
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen
# df=X, label=y, size=80,n_feat=number of column in X,n_parents=64, mutation_rate=0.2,n_gen=5
def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen,X_train,Y_train,X_test,Y_test)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo.copy(),best_score.copy()
def split(df,label):
    X_tr, X_te, Y_tr, Y_te = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_tr, X_te, Y_tr, Y_te
    
def genAlgo_v2(data,label='class'):
    y=data['class']
    X=data.drop('class',axis=1)
    X_train,X_test, Y_train, Y_test = split(X,y)
    chromo_df_bc,score_bc=generations(X,y,size=40,n_feat=X.shape[1],n_parents=32,mutation_rate=0.20,n_gen=3,
                         X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)
    score_bc, chromo_df_bc = np.array(score_bc), np.array(chromo_df_bc) 
    inds = np.argsort(score_bc)                                    
    score_bc=list(score_bc[inds][::-1])
    chromo_df_bc=list(chromo_df_bc[inds,:][::-1]) 
    #score_bc=list(score_bc)
    #chromo_df_bc=list(chromo_df_bc) 
    X=X.iloc[:,chromo_df_bc[0]]
    selectedFeatures=list(X.columns)
    data_combined = pd.concat([X, y], axis=1)
    genRes=storeData(score_bc[0] , selectedFeatures, chromo_df_bc[0],data_combined)
    return genRes;  

In [6]:

Data = pd.read_csv("onlyAlpha.csv")
# Data

In [7]:
store=genAlgo_v2(Data.copy(),label='class')


Best score in generation 1 : [0.8881118881118881]
Best score in generation 2 : [0.9090909090909091]
Best score in generation 3 : [0.8881118881118881]


In [8]:
# print("best chromosome : "+chromo_df_bc)
# print("best score: "+score_bc)
store.data

,O1-LE_alpha,T3-LE_alpha,Fp2-LE_alpha,F8-LE_alpha,T4-LE_alpha,Cz-LE_alpha,Pz-LE_alpha,A2-A1_alpha,class
0,6.69,6.46,11.32,10.04,2.77,8.54,9.02,7.85,0
1,11.09,6.73,14.67,16.58,4.62,11.43,12.49,9.74,0
2,8.80,4.70,8.55,6.04,2.65,10.43,9.40,8.79,0
3,17.34,14.99,28.18,18.37,5.11,26.13,36.55,7.69,0
4,15.42,10.13,11.47,8.55,2.64,16.71,26.95,6.90,0
...,...,...,...,...,...,...,...,...,...
1136,16.63,24.31,28.56,31.56,42.09,32.34,24.29,43.66,1
1137,18.11,43.28,34.67,29.04,26.23,39.78,30.92,21.07,1
1138,13.41,13.71,25.51,14.65,10.93,19.92,16.35,4.84,1
1139,19.03,35.75,16.07,10.88,12.91,25.57,22.33,9.83,1


In [9]:
store.features

['O1-LE_alpha',
 'T3-LE_alpha',
 'Fp2-LE_alpha',
 'F8-LE_alpha',
 'T4-LE_alpha',
 'Cz-LE_alpha',
 'Pz-LE_alpha',
 'A2-A1_alpha']

In [10]:
store.result

0.9090909090909091

In [11]:
store.cromosome

array([False, False, False, False,  True, False,  True, False, False,
        True, False, False, False, False,  True,  True, False,  True,
        True,  True])

In [65]:

# allF=Data.columns.tolist()

# indices_to_select = [1, 3, 4]

# filFeature = [allF[i] for i in indices_to_select]
# filFeature.append('class')
# Data=Data[filFeature]
# Data

In [6]:
res=quick_result(Data.copy(),class_name="class",test_size=0.20,random_state=42)
res

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.663755,0.144444,1.000000,0.252427
1,LogisticRegression,0.855895,0.833333,0.870504,0.819672
2,DecissionTree,0.842795,0.855556,0.834532,0.810526
3,RandomForest,0.890830,0.900000,0.884892,0.866310
4,Voting,0.864629,0.700000,0.971223,0.802548


In [7]:
corrOutput=correlationResult(Data.copy(),Type=0)
firstCromo=corrOutput.cromosome

In [8]:
firstCromo

[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [9]:
corrOutput.result

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.641921,0.111111,0.985612,0.196078
1,LogisticRegression,0.620087,0.266667,0.848921,0.355556
2,DecissionTree,0.724891,0.766667,0.697842,0.686567
3,RandomForest,0.799127,0.811111,0.791367,0.760417
4,Voting,0.663755,0.188889,0.971223,0.306306


In [10]:
corrOutput1=correlationResult(Data.copy(),Type=1)
secondCromo=corrOutput1.cromosome

T5-LE_alpha 0.6052677146197165
O1-LE_alpha 0.5839896523372571
P3-LE_alpha 0.6412827205109085


In [11]:
secondCromo

[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [12]:
corrOutput1.result

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.646288,0.111111,0.992806,0.198020
1,LogisticRegression,0.563319,0.288889,0.741007,0.342105
2,DecissionTree,0.742358,0.700000,0.769784,0.681081
3,RandomForest,0.812227,0.755556,0.848921,0.759777
4,Voting,0.650655,0.200000,0.942446,0.310345


In [13]:
corrOutput2=correlationResult(Data.copy(),Type=2)
thirdCromo=corrOutput2.cromosome

In [14]:
thirdCromo

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

In [15]:
corrOutput2.result

,Algo Names,Accuracy,Sensitivity,Specificity,F1-Score
0,SVM,0.637555,0.077778,1.000000,0.144330
1,LogisticRegression,0.537118,0.511111,0.553957,0.464646
2,DecissionTree,0.807860,0.744444,0.848921,0.752809
3,RandomForest,0.855895,0.811111,0.884892,0.815642
4,Voting,0.724891,0.366667,0.956835,0.511628


In [16]:
# finalFeature,finalCromo,res=GeneticAlgorithm_singleCromosomePass(Data.copy(),Cromosome=firstCromo,class_name='class')
# finalFeature

In [26]:
mainRes=GeneticAlgo(Data.copy(),class_name='class')
mainRes.result

single cromo : 
None
Default Running...
20

GENERATION: #: 1

Family: #: 1

FV for Mutated Child #1 at Gen 1 : 0.4067762200260477
FV for Mutated Child #2 at Gen 1 : 0.5590745422508234

Family: #: 2

FV for Mutated Child #1 at Gen 1 : 0.8308128399601624
FV for Mutated Child #2 at Gen 1 : 0.5758254807323987

Family: #: 3

FV for Mutated Child #1 at Gen 1 : 0.4479506626829082
FV for Mutated Child #2 at Gen 1 : 0.587171531448709

Family: #: 4

FV for Mutated Child #1 at Gen 1 : 0.6151842488316862
FV for Mutated Child #2 at Gen 1 : 0.7099977016777752

Family: #: 5

FV for Mutated Child #1 at Gen 1 : 0.5951505401057229
FV for Mutated Child #2 at Gen 1 : 0.7992377231287826
New_Population shape: (10, 20)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
mainRes.features

In [ ]:
custom1=GeneticAlgo(Data.copy(),singleCromosome=firstCromo,class_name='class')
custom1.result

In [ ]:
custom1.features

In [ ]:
custom2=GeneticAlgo(Data.copy(),singleCromosome=secondCromo,class_name='class')
custom2.result

In [ ]:
custom2.features

In [ ]:
custom3=GeneticAlgo(Data.copy(),singleCromosome=thirdCromo,class_name='class')
custom3.result

In [ ]:
custom3.features